In [ ]:
from models.unet import UNet

class DDPM(nn.Module):
    def __init__(self, model, beta_schedule):
        super().__init__()
        self.model = model
        self.beta_schedule = beta_schedule  # schedule based on schedule

    def forward(self, x_t, t):
        return self.model(x_t, t)  # Predict noise

    def sample(self, num_samples):
        # can be deterministic or statistic
        pass
